<a href="https://colab.research.google.com/github/CantonCode/Logistic-Regression-Assignment/blob/main/16164571_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pathlib
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, SeparableConv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import History 
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os
import pandas as pd
import glob
import re

print(os.getcwd())

C:\Users\conor\Desktop\4th year\Untitled Folder


Reading in my csv files of maps. I am using regex search to find the id of each map and I add these too a labels array

I create 5 channels for each of the map features create an array of 5 channels for each of the maps. 

map shape will be (49,49,5)

In [ ]:
mapDirectory = (r"C:\Users\conor\Desktop\4th year\Untitled Folder\maps")
mapFiles = []
channel1,channel2,channel3,channel4,channel5 = [],[],[],[],[]
labels = []
for file in os.listdir(mapDirectory):
    fullPath = os.path.join(mapDirectory, file)
    df = pd.read_csv(fullPath).values.reshape(49, 49)
    mapFiles.append(df)
    labels.append(re.search('map_(.+?).csv', file).group(1))
    
    ch1 = np.where(df != 10,0,df)
    channel1.append(ch1)
    
    
    ch2 = np.where(df != 1,0,df)
    channel2.append(ch1)
    
    ch3 = np.where(df != 2,0,df)
    channel3.append(ch1)
    
    ch4 = np.where(df != 8,0,df)
    channel4.append(ch1)
    
    ch5 = np.where(df != 9,0,df)
    channel5.append(ch1)
    
    
    
    
mapFiles = np.array(mapFiles) 
channel1 = np.array(channel1)
channel2 = np.array(channel2)
channel3 = np.array(channel3)
channel4 = np.array(channel4)
channel5 = np.array(channel5)

Printing out the shapes of the arrays to ensure they are correct

In [ ]:
print(mapFiles.shape)
print(channel1.shape)
print(channel2.shape)
print(channel3.shape)
print(channel4.shape)
print(channel5.shape)

(14665, 49, 49)
(14665, 49, 49)
(14665, 49, 49)
(14665, 49, 49)
(14665, 49, 49)
(14665, 49, 49)


Append all the channels together onto one single array for each map

In [ ]:
maps = []
maps.append([channel1,channel2,channel3,channel4,channel5])
maps = np.array(maps)
maps = maps.reshape(len(mapFiles),49,49,5)
maps.shape

(14665, 49, 49, 5)

In [ ]:
len(labels)

14665

read in the players files and data

In [ ]:
playerDir = (r"C:\Users\conor\Desktop\4th year\Untitled Folder\players")
playerFiles = []
for file in os.listdir(playerDir):
    fullPath = os.path.join(playerDir, file)
    df = pd.read_csv(fullPath, dtype = str)
    playerFiles.append(df)

In [ ]:
playerFiles[0]

,MapId,Win Rate,Loss Rate,Draw Rate
0,338715339,289,219,492
1,978974935,208,87,705
2,728758576,165,357,478
3,204725116,185,122,693
4,1901294908,4,15,981
5,1567030292,250,589,161
6,2029429188,496,488,16
7,33878278,7,37,956
8,225769727,39,18,943
9,1927023283,827,103,70


In [ ]:
len(playerFiles)

2217

Preformed some pre processing of the player data which removes any empty files from the player data

In [ ]:
playerDf = pd.concat(playerFiles, axis = 0, ignore_index = True)
playerDf.drop(playerDf.filter(regex="Unnamed"), axis=1, inplace=True)
playerDf.head()

,MapId,Win Rate,Loss Rate,Draw Rate
0,338715339,289,219,492
1,978974935,208,87,705
2,728758576,165,357,478
3,204725116,185,122,693
4,1901294908,4,15,981


we iterate through the each row of data in the player data field.

we calculate if the player won or lost 

if the win rate is higher than the loss rate and the draw rate they won and it is assigned a 0

the the loss rate is higher than the win rate and the draw rate we assign it a 1 a

the draw rate is higher than the win or loss rate then we assign it a 2

In [ ]:
for i, row in playerDf.iterrows():
    if(int(row['Win Rate']) > int(row['Loss Rate'])) and (int(row['Win Rate']) > int(row['Draw Rate'])):
        playerDf.at[i, 'Outcome']=0 
    elif (int(row['Win Rate']) < int(row['Loss Rate'])) and (int(row['Loss Rate']) > int(row['Draw Rate'])):
        playerDf.at[i, 'Outcome']=1 
    else:
        playerDf.at[i, 'Outcome']=2 

we filter out any of the player data where they drew as are just looking to predict a win or a loss not a draw

In [ ]:
playerDf = playerDf[playerDf.Outcome != 2]
playerDf.head

<bound method NDFrame.head of            MapId Win Rate Loss Rate Draw Rate  Outcome
5     1567030292      250       589       161      1.0
6     2029429188      496       488        16      0.0
9     1927023283      827       103        70      0.0
10     853603721      566       309       125      0.0
11    1954917924      565       425        10      0.0
...          ...      ...       ...       ...      ...
8950   387999470      276       133       127      0.0
8952  2008057506      409       560        31      1.0
8953  1875012101      357       408       235      1.0
8954   568706823      109       145        11      1.0
8955  1597566094        3         1         0      0.0

[6809 rows x 5 columns]>

In [ ]:
len(playerDf)

6809

In [ ]:
maps = maps[:len(playerDf)]
X = maps
y = playerDf['Outcome']
print(len(maps))
print(len(y))

6809
6809


In [ ]:
def build_model():      
        input_img = Input(shape=(49,49,5), name='ImageInput')
        x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_1')(input_img)
        x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_2')(x)
        x = MaxPooling2D((2,2), name='pool1')(x)
        
        x = Flatten(name='flatten')(x)
        x = Dense(1024, activation='relu', name='fc1')(x)
        x = Dropout(0.7, name='dropout1')(x)
        x = Dense(512, activation='relu', name='fc2')(x)
        x = Dropout(0.5, name='dropout2')(x)
        
        x = Dense(1, activation='sigmoid', name='fc3')(x)
        
        model = Model(inputs=input_img, outputs=x)
        return model

In [ ]:
model = build_model()
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ImageInput (InputLayer)      [(None, 49, 49, 5)]       0         
_________________________________________________________________
Conv1_1 (Conv2D)             (None, 49, 49, 64)        2944      
_________________________________________________________________
Conv1_2 (Conv2D)             (None, 49, 49, 64)        36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 24, 24, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
fc1 (Dense)                  (None, 1024)              37749760  
_________________________________________________________________
dropout1 (Dropout)           (None, 1024)              0   

In [ ]:
base_learning_rate = 0.001
model.compile(optimizer = tf.keras.optimizers.Adam(lr = base_learning_rate),
             loss = tf.keras.losses.MeanSquaredError(),
             metrics = ['accuracy','mean_absolute_error', tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
def scheduler(epoch, lr):
    if epoch < 3:
        return lr
    else:
        return lr / 5 
    
scheduler_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)

In [ ]:
history = History()

In [ ]:
model.fit(X_train, y_train, epochs = 10, callbacks = [scheduler_callback, early_stopping_callback,history],  
          validation_data=(X_test,y_test))


Epoch 1/10
149/149 [==============================] - 40s 269ms/step - loss: 0.2191 - accuracy: 0.6416 - mean_absolute_error: 0.4446 - root_mean_squared_error: 0.4681 - val_loss: 0.2437 - val_accuracy: 0.5835 - val_mean_absolute_error: 0.4701 - val_root_mean_squared_error: 0.4936
Epoch 2/10
149/149 [==============================] - 40s 266ms/step - loss: 0.2018 - accuracy: 0.6815 - mean_absolute_error: 0.4128 - root_mean_squared_error: 0.4492 - val_loss: 0.2671 - val_accuracy: 0.5482 - val_mean_absolute_error: 0.4689 - val_root_mean_squared_error: 0.5168
Epoch 3/10
149/149 [==============================] - 39s 262ms/step - loss: 0.1825 - accuracy: 0.7197 - mean_absolute_error: 0.3693 - root_mean_squared_error: 0.4272 - val_loss: 0.2832 - val_accuracy: 0.5374 - val_mean_absolute_error: 0.4739 - val_root_mean_squared_error: 0.5322
Epoch 4/10
149/149 [==============================] - 39s 264ms/step - loss: 0.1585 - accuracy: 0.7673 - mean_absolute_error: 0.3404 - root_mean_squared_erro

In [ ]:
predictions = model.predict(X_test)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,accuracy,mean_absolute_error,root_mean_squared_error,val_loss,val_accuracy,val_mean_absolute_error,val_root_mean_squared_error,lr,epoch
0,0.219120,0.641628,0.444566,0.468103,0.243669,0.583456,0.470109,0.493628,0.00020,0
1,0.201816,0.681494,0.412841,0.449239,0.267100,0.548213,0.468925,0.516818,0.00020,1
2,0.182481,0.719681,0.369347,0.427177,0.283235,0.537445,0.473881,0.532198,0.00020,2
3,0.158471,0.767310,0.340445,0.398085,0.294695,0.531082,0.474337,0.542858,0.00004,3


we augment the data by usuing the np.flipud to flip the map data and also np.roll to shift the data by 2 posistions

I also copy the player data and add it onto the end of the player data that already exists

In [ ]:
temp = []
for amap in maps:
    amap =  np.roll(amap,2)
    temp.append(np.array(np.flipud(amap)))
    
arr = np.array(temp)
aug_map_data = np.concatenate((maps, arr))

playerDfCopy = playerDf.copy()
aug_player_data = pd.concat([playerDf,playerDfCopy])



len(aug_map_data)

In [ ]:
augX = aug_map_data
augy = aug_player_data['Outcome']
print(len(augX))
print(len(augy))

13618
13618


In [ ]:
augX_train, augX_test, augy_train, augy_test = train_test_split(augX, augy, test_size = 0.3, random_state = 40)

In [ ]:
augModel = build_model()

In [ ]:
augModel.compile(optimizer = tf.keras.optimizers.Adam(lr = base_learning_rate),
             loss = tf.keras.losses.MeanSquaredError(),
             metrics = ['accuracy','mean_absolute_error', tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
history2 = History()

In [ ]:
augModel.fit(augX_train, augy_train, epochs = 10, callbacks = [scheduler_callback, early_stopping_callback,history2],  
          validation_data=(augX_test,augy_test))

Epoch 1/10
298/298 [==============================] - 81s 271ms/step - loss: 0.2352 - accuracy: 0.6169 - mean_absolute_error: 0.4699 - root_mean_squared_error: 0.4850 - val_loss: 0.2393 - val_accuracy: 0.6104 - val_mean_absolute_error: 0.4747 - val_root_mean_squared_error: 0.4892
Epoch 2/10
298/298 [==============================] - 79s 266ms/step - loss: 0.2333 - accuracy: 0.6172 - mean_absolute_error: 0.4667 - root_mean_squared_error: 0.4830 - val_loss: 0.2411 - val_accuracy: 0.6094 - val_mean_absolute_error: 0.4793 - val_root_mean_squared_error: 0.4910
Epoch 3/10
298/298 [==============================] - 82s 274ms/step - loss: 0.2285 - accuracy: 0.6240 - mean_absolute_error: 0.4580 - root_mean_squared_error: 0.4780 - val_loss: 0.2465 - val_accuracy: 0.5886 - val_mean_absolute_error: 0.4758 - val_root_mean_squared_error: 0.4965
Epoch 4/10
298/298 [==============================] - 84s 282ms/step - loss: 0.2163 - accuracy: 0.6546 - mean_absolute_error: 0.4408 - root_mean_squared_erro

In [ ]:
his = pd.DataFrame(history2.history)
his['epoch'] = history.epoch
his.tail()

,loss,accuracy,mean_absolute_error,root_mean_squared_error,val_loss,val_accuracy,val_mean_absolute_error,val_root_mean_squared_error,lr,epoch
0,0.235179,0.616870,0.469902,0.484953,0.239330,0.610377,0.474686,0.489214,0.00020,0
1,0.233253,0.617184,0.466715,0.482963,0.241103,0.609398,0.479298,0.491022,0.00020,1
2,0.228485,0.624003,0.457964,0.478001,0.246465,0.588595,0.475774,0.496452,0.00020,2
3,0.216261,0.654637,0.440791,0.465039,0.251740,0.573911,0.473065,0.501737,0.00004,3
